# Generating and Interpreting Logo Representations Learned by StyleGAN2

Welcome to our Spring 2021 Junior Paper for Princeton University!

There are many difficulties associated with logo design, including client to designer communication, creative roadblocks, and plagiarism / copyright laws. To solve some of these problems, we created a tool that allows users to generate and control features of logos that *do not exist*.

We used StyleGAN2 to train on two different datasets - one of logos with text and one of logos without text. We then carefully interpreted the latent space of each model and implemented other helpful features.

Contact us if you have any feedback or questions!



*   Arin Champati: champati@princeton.edu
*   Brendan Houle: bhoule@princeton.edu

# Starting Instructions

###1. Click on "Runtime" -> "Change runtime type" -> Select "GPU" from the "Hardware Accelerator" dropdown and save.

###2. Select a model from one of the following dropdowns, and start making logos!

#Logos without Text
This model generates brand marks, or logos without text.

In [ ]:
#@title Download Requirements
print("Downloading Model")
print()
!gdown --id 1NgE9p_rrC9oj_rggi-JmpftjW8Wens4R -O /content/model.pkl

print()
print("Loading Directions")
print()
!gdown --id 105_Y5r5Vn3uhOAR0o6NNb4yR7xlmGA92 -O /content/directions.pkl

print()
print("Loading Projected Logos")
print()
# # stylegan2 projection with absolute difference as loss
# !gdown --id 1-1_WLmqo7VCXFIxNIltEkgrRIkPNzGpc -O /content/existing.pkl

# # regular stylegan2 projection
# !gdown --id 1-3mdl3GiY--2D15dtgwFSBPvfBH-lAPa -O /content/existing.pkl

# best from both 
!gdown --id 1-ZV85AuHQeJtR-dizLSXPQqEHOTJPc3G -O /content/existing.pkl

print()
print("Downloading Labeled Seeds")
print()
!gdown --id 11HaNuy0M5Tds6yY5bMXFlhFyDAO0VTxe -O /content/seeds_dict.pkl

import pickle
with open("/content/seeds_dict.pkl", "rb") as f:
  seeds_dict = pickle.load(f)

print()
print("Cloning Custom StyleGAN2 Repository")
print()
#@title Load In Model
!git clone https://github.com/arin-champati/stylegan2
%cd stylegan2/
import sys, os
%tensorflow_version 1.x

import pretrained_networks
import dnnlib
from dnnlib import tflib
from matplotlib import pyplot as plt

from pathlib import Path
from PIL import Image
import pickle
import numpy as np
from IPython.display import clear_output

import ipywidgets as widgets
from tqdm import tqdm
import tensorflow as tf
import random
TRUNC_VAL = 0.7
def linear_interpolate(code1, code2, alpha):
    return code1 * alpha + code2 * (1 - alpha)

def linear_interpolate_w(w1, w2, shape_alpha, color_alpha):
  w1 = w1.squeeze()
  w2 = w2.squeeze()
  result_w = np.zeros(w1.shape)

  # shape features
  for layer in range(0, 8):
      result_w[layer] = w2[layer] * shape_alpha + w1[layer] * (1 - shape_alpha)
  
  for layer in range(8, 12):
      result_w[layer] = w2[layer] * color_alpha + w1[layer] * (1 - color_alpha)

  return result_w

def generate_neighbors_w(og_w, diameter, num_samples, start, end):
    ws = []
    ws.append(og_w)
    for s in range(num_samples):
        random = np.random.uniform(-diameter,diameter,[12,512])
        w = og_w.copy()
        for layer in range(start, end):
          w[layer] += random[layer]

        ws.append(w)
    return ws

def generate_neighbors(og_z, diameter, num_samples):
    zs = []
    zs.append(og_z)
    for s in range(num_samples):
        # np.random.seed()
        random = np.random.uniform(-diameter,diameter,[1,512])
        zs.append(np.clip(np.add(og_z,random),-1,1))
    return zs

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols
    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid
    
# Generate images given a random seed (Integer)
def generate_image_random(rand_seed):
    rnd = np.random.RandomState(rand_seed)
    z = rnd.randn(1, *Gs.input_shape[1:])
    tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars})
    images = Gs.run(z, None, **Gs_kwargs)
    return images[0], z

# Generate images given a latent code ( vector of size [1, 512] )
def generate_image_from_z(z):
    images = Gs.run(z, None, **Gs_kwargs)
    return images[0]
def z_to_w(z):
    w = Gs.components.mapping.run(z, None)[0]
    return w
def truncate(dlatents, truncation_psi, maxlayer = 12):
    dlatent_avg = tf.get_default_session().run(Gs.own_vars["dlatent_avg"])
    layer_idx = np.arange(12)[np.newaxis, :, np.newaxis]
    ones = np.ones(layer_idx.shape, dtype=np.float32)
    coefs = tf.where(layer_idx < maxlayer, truncation_psi * ones, ones)
    return tf.get_default_session().run(tflib.lerp(dlatent_avg, dlatents, coefs))

# Code to load the StyleGAN2 Model
def generate_image_from_w(dlatents, truncation_psi = TRUNC_VAL, randomize_noise = False):
    if not truncation_psi is None:
        dlatents_trunc = truncate(dlatents, truncation_psi)
    else:
        dlatents_trunc = dlatents
        
    # Run the network
    fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    result_image = Gs.components.synthesis.run(
        dlatents_trunc.reshape((-1, 12, 512)),
        randomize_noise = randomize_noise,
        minibatch_size = 1,
        output_transform=fmt
    )[0]
    return result_image
def load_model():
    _G, _D, Gs = pretrained_networks.load_networks('/content/model.pkl')
    
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.truncation_psi = 0.7
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False    
    return Gs, noise_vars, Gs_kwargs
results_size = 128
Gs, noise_vars, Gs_kwargs = load_model()

good_seeds = [391049438, 812988239, 864684474, 169869150, 1821476060, 1736720173, 1388122245, 470455603, 109205828, 
              1466675128, 810863819, 172582121, 227593680, 297356458, 350606507,
              1428287491, 843, 145, 5310, 3967, 389746, 2124813335, 1444062920, 1651583255,
              715622503, 130814676, 57, 20, 670, 3345, 33457, 137, 138, 812988233, 32353, 8469015,
              1723225816, 196525129, 57873533, 1513149282, 1967834917, 418002533, 1442228425, 1193347417,
              130029044, 198562280, 1093333447, 1611767532, 1556421108, 684367779, 1922093669, 
              1370174848, 1540350065, 1706219851, 537009070, 908536436, 1716616451, 295283057, 842246229,
              1356768744, 195488666, 1294954460, 2502608, 349442220, 740312126, 1252333816, 1021847599, 
              1113669133, 1917592453, 1052444511, 446373871, 125393091, 1198508016, 1399116885, 1224077971, 
              1726114719, 994796117, 1306037308, 1860705326, 1259247846, 872237283, 889028859, 68870, 
              6163011, 9761915, 2316994, 4406784, 1478901, 1099883, 5762883, 669401, 8233848, 
              1411392, 9721645, 7909081, 6627487, 4256231, 7437617, 
              ]

from matplotlib.gridspec import SubplotSpec
def create_subtitle(fig: plt.Figure, grid: SubplotSpec, title: str):
    "Sign sets of subplots with title"
    row = fig.add_subplot(grid)
    # the '\n' is important
    row.set_title(f'{title}\n', fontweight='semibold')
    # hide subplot
    row.set_frame_on(False)
    row.axis('off')

print()
clear_output()
print("Ready!")

In [ ]:
#@title Set Random Seed if Desired (For Reproducibility)
seed = int(input('Random Seed: '))
np.random.seed(seed)

In [ ]:
#@title Select Method to Choose First Logo
logo_method_1 = "Good" #@param ["Random", "Good", "By Feature", "Existing"]


In [ ]:
#@title Generate First Logo
if logo_method_1 == 'Random':
  TRUNC_VAL = 0.7
  rand_seed = np.random.randint(0,1000000000)
  images1, z1 = generate_image_random(rand_seed)
  w1 = z_to_w(z1)
  images1 = generate_image_from_z(z1)
  image1 = Image.fromarray(images1).resize((results_size, results_size))
  display(image1)
elif logo_method_1 == 'Good':
  TRUNC_VAL = 0.7
  rand_seed = np.random.choice(good_seeds)
  images1, z1 = generate_image_random(rand_seed)
  w1 = z_to_w(z1)
  images1 = generate_image_from_z(z1)
  image1 = Image.fromarray(images1).resize((results_size, results_size))
  display(image1)
elif logo_method_1 == 'Existing':
  TRUNC_VAL = 1
  pkl_file = open('/content/existing.pkl', 'rb')
  existing_logos = pickle.load(pkl_file)
  pkl_file.close()

  print("Disclaimer: using existing logos provides noisy and unpredictable results later on")
  print("Method works best if both logos are existing logos")
  print()
  print("Options: ")
  per_row = 5
  num_strings = (len(existing_logos) // per_row) + 1
  for i in range(num_strings):
    intermediate_str = ', '.join(list(sorted(existing_logos.keys()))[i*num_strings:(i+1)*num_strings])  
    print(intermediate_str)

  print()
  key = input("Type name of logo from above options: ")
  if key in sorted(existing_logos.keys()):
    w1 = existing_logos[key]
    images1 = generate_image_from_w(w1,truncation_psi=TRUNC_VAL)
    image1 = Image.fromarray(images1).resize((results_size, results_size))
    
    print()
    display(image1)
  else:
    print()
    print('Error: Enter a valid logo name')
elif logo_method_1 == 'Your Own':
  print(4)
elif logo_method_1 == 'By Feature':
  TRUNC_VAL = 0.7
  print("Options: ")
  per_row = 3
  num_strings = (len(seeds_dict) // per_row) + 1
  for i in range(num_strings):
    key_names = list(seeds_dict.keys())
    key_names.remove('all')
    intermediate_str = ', '.join(key_names[i*num_strings:(i+1)*num_strings])  
    print(intermediate_str)
  
  key = input("Type name of feature from above options: ")
  if key in seeds_dict:
    seeds = seeds_dict[key]
    seed = np.random.choice(seeds)

    images1, z1 = generate_image_random(seed)
    w1 = z_to_w(z1)

    images1 = generate_image_from_z(z1)
    image1 = Image.fromarray(images1).resize((results_size, results_size))
    
    print()
    display(image1)
  else:
    print()
    print('Error: Enter a valid feature name')

w1 = w1.squeeze()

In [ ]:
from matplotlib import pyplot as plt
#@title Generate Similar Logos  
num_samples = 8
if logo_method_1 == 'Existing':
  neighbors1 = generate_neighbors_w(w1, diameter=3, num_samples=8, start=0, end=5)
  neighbors2 = generate_neighbors_w(w1, diameter=4, num_samples=8, start=8, end=12)
  neighbors3 = generate_neighbors_w(w1, diameter=2.5, num_samples=num_samples, start=0, end=12)
else:
  neighbors1 = generate_neighbors_w(w1, diameter=5, num_samples=8, start=0, end=5)
  neighbors2 = generate_neighbors_w(w1, diameter=5, num_samples=8, start=8, end=12)
  neighbors3 = generate_neighbors(z1, diameter=.5, num_samples=num_samples)

f,ax = plt.subplots(3,len(neighbors1),figsize=(12,5))

all_neighbors_1 = [neighbors1, neighbors2, neighbors3]
count = 0
row_col = []
for i, neighbors in enumerate([neighbors1, neighbors2, neighbors3]):
  for j, im in enumerate(neighbors):
    if i == 2:
      if logo_method_1 == 'Existing':
        ax[i, j].imshow(generate_image_from_w(im, truncation_psi=TRUNC_VAL))
      else:
         ax[i, j].imshow(generate_image_from_z(im))
    else:
      ax[i, j].imshow(generate_image_from_w(im, truncation_psi=TRUNC_VAL))

    ax[i, j].title.set_text(count)
    ax[i, j].axis('off')
    row_col.append((i, j))
    count += 1

grid = plt.GridSpec(3, len(neighbors1))
create_subtitle(f, grid[0, ::], 'Varying Shape')
create_subtitle(f, grid[1, ::], 'Varying Color')
create_subtitle(f, grid[2, ::], 'Varying Both')
f.tight_layout()
plt.show()

In [ ]:
#@title Select First Logo from Above Choices
first_index = int(input('Which logo would you like to proceed with? (Enter a number): '))
row = row_col[first_index][0]
col = row_col[first_index][1]

w1 = all_neighbors_1[row][col]
if w1.shape[0] != 12:
  w1 = z_to_w(w1)

images1 = generate_image_from_w(w1,truncation_psi=TRUNC_VAL).squeeze()
image1 = Image.fromarray(images1).resize((results_size, results_size))
print()
display(image1)

In [ ]:
#@title Select Method to Choose Second Logo
logo_method_2 = "Good" #@param ["Random", "Good", "By Feature", "Existing"]


In [ ]:
#@title Generate Second Logo
#@title Generate First Logo
if logo_method_2 == 'Random':
  TRUNC_VAL = 0.7
  rand_seed = np.random.randint(0,1000000000)
  images2, z2 = generate_image_random(rand_seed)
  w2 = z_to_w(z2)
  images2 = generate_image_from_z(z2)
  image2 = Image.fromarray(images2).resize((results_size, results_size))
  display(image2)
elif logo_method_2 == 'Good':
  TRUNC_VAL = 0.7
  rand_seed = np.random.choice(good_seeds)
  images2, z2 = generate_image_random(rand_seed)
  w2 = z_to_w(z2)
  images2 = generate_image_from_z(z2)
  image2 = Image.fromarray(images2).resize((results_size, results_size))
  display(image2)
elif logo_method_2 == 'Existing':
  TRUNC_VAL = 1
  pkl_file = open('/content/existing.pkl', 'rb')
  existing_logos = pickle.load(pkl_file)
  pkl_file.close()

  print("Disclaimer: using existing logos provides noisy and unpredictable results later on")
  print("Method works best if both logos are existing logos")
  print()

  print("Options: ")
  per_row = 5
  num_strings = (len(existing_logos) // per_row) + 1
  for i in range(num_strings):
    intermediate_str = ', '.join(list(sorted(existing_logos.keys()))[i*num_strings:(i+1)*num_strings])  
    print(intermediate_str)

  print()
  key = input("Type name of logo from above options: ")
  if key in sorted(existing_logos.keys()):
    w2 = existing_logos[key]
    images2 = generate_image_from_w(w2,truncation_psi=TRUNC_VAL).squeeze()
    image2 = Image.fromarray(images2).resize((results_size, results_size))
    
    print()
    display(image2)
  else:
    print()
    print('Error: Enter a valid logo name')
elif logo_method_2 == 'By Feature':
  TRUNC_VAL = 0.7
  print("Options: ")
  per_row = 3
  num_strings = (len(seeds_dict) // per_row) + 1
  for i in range(num_strings):
    key_names = list(seeds_dict.keys())
    key_names.remove('all')
    intermediate_str = ', '.join(key_names[i*num_strings:(i+1)*num_strings])  
    print(intermediate_str)
  
  key = input("Type name of feature from above options: ")
  if key in seeds_dict:
    seeds = seeds_dict[key]
    seed = np.random.choice(seeds)

    images2, z2 = generate_image_random(seed)
    w2 = z_to_w(z2)

    images2 = generate_image_from_z(z2)
    image2 = Image.fromarray(images2).resize((results_size, results_size))
    
    print()
    display(image2)
  else:
    print()
    print('Error: Enter a valid feature name')
else:
  print(4)

w2 = w2.squeeze()

In [ ]:
from matplotlib import pyplot as plt
#@title Generate Similar Logos
if logo_method_2 == 'Existing':
  neighbors1 = generate_neighbors_w(w2, diameter=3, num_samples=8, start=0, end=5)
  neighbors2 = generate_neighbors_w(w2, diameter=4, num_samples=8, start=8, end=12)
  neighbors3 = generate_neighbors_w(w2, diameter=2.5, num_samples=num_samples, start=0, end=12)
else:
  neighbors1 = generate_neighbors_w(w2, diameter=5, num_samples=8, start=0, end=5)
  neighbors2 = generate_neighbors_w(w2, diameter=5, num_samples=8, start=8, end=12)
  neighbors3 = generate_neighbors(z2, diameter=.5, num_samples=num_samples)

f,ax = plt.subplots(3,len(neighbors1),figsize=(12,5))

all_neighbors_2 = [neighbors1, neighbors2, neighbors3]
count = 0
row_col = []
for i, neighbors in enumerate([neighbors1, neighbors2, neighbors3]):
  for j, im in enumerate(neighbors):
    if i == 2:
      if logo_method_2 == 'Existing':
        ax[i, j].imshow(generate_image_from_w(im, truncation_psi=TRUNC_VAL))
      else:
         ax[i, j].imshow(generate_image_from_z(im))
    else:
      ax[i, j].imshow(generate_image_from_w(im, truncation_psi=TRUNC_VAL))

    ax[i, j].title.set_text(count)
    ax[i, j].axis('off')
    row_col.append((i, j))
    count += 1

grid = plt.GridSpec(3, len(neighbors1))
create_subtitle(f, grid[0, ::], 'Varying Shape')
create_subtitle(f, grid[1, ::], 'Varying Color')
create_subtitle(f, grid[2, ::], 'Varying Both')
f.tight_layout()
plt.show()

In [ ]:
#@title Select Second Logo from Above Choices
first_index = int(input('Which logo would you like to proceed with? (Enter a number): '))
row = row_col[first_index][0]
col = row_col[first_index][1]

w2 = all_neighbors_2[row][col]
if w2.shape[0] != 12:
  w2 = z_to_w(w2)

images2 = generate_image_from_w(w2,truncation_psi=TRUNC_VAL).squeeze()
image2 = Image.fromarray(images2).resize((results_size, results_size))
display(image2)

In [ ]:
#@title Mix Styles

images1 = generate_image_from_w(w1,truncation_psi=TRUNC_VAL).squeeze()
image1 = Image.fromarray(images1).resize((results_size, results_size))
images2 = generate_image_from_w(w2,truncation_psi=TRUNC_VAL).squeeze()
image2 = Image.fromarray(images2).resize((results_size, results_size))

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
def f(shape_alpha, color_alpha):
    images = generate_image_from_w(linear_interpolate_w(w1, w2, shape_alpha, color_alpha),truncation_psi=TRUNC_VAL)
    images = Image.fromarray(images).resize((results_size, results_size))
    im_grid = image_grid([image1, images, image2], 1, 3)
    return im_grid
shape_alpha = widgets.FloatSlider(min=0, max=1, step=.05, value=0, description= 'shape', continuous_update=False)
color_alpha = widgets.FloatSlider(min=0, max=1, step=.05, value=0, description= 'color',  continuous_update=False)
current_im = interact(f, shape_alpha=shape_alpha, color_alpha=color_alpha);

In [ ]:
#@title Modify Features
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

latent_code_to_use = linear_interpolate_w(w1, w2, shape_alpha.value, color_alpha.value)
pkl_file = open('/content/directions.pkl', 'rb')
latent_controls = pickle.load(pkl_file)
pkl_file.close()

for key in latent_controls:
  latent_controls[key][3] = np.squeeze(latent_controls[key][3])
latent_code_to_use = np.squeeze(latent_code_to_use)
result_image = generate_image_from_w(latent_code_to_use,TRUNC_VAL)
result_image = Image.fromarray(result_image)

def apply_latent_controls(self):
    
    image_outputs = controller.children[0]
    feature_sliders = controller.children[1]
    
    slider_hboxes = feature_sliders.children[:-2]
    latent_movements = [(x.children[1].value, x.children[0].value) for x in slider_hboxes]

    modified_latent_code = np.array(latent_code_to_use)
    for feature, amount_to_move in latent_movements:
      for layer in range(latent_controls[feature][0],latent_controls[feature][1]):
        modified_latent_code[layer] += latent_controls[feature][3]*amount_to_move*latent_controls[feature][2]

    images = generate_image_from_w(modified_latent_code, truncation_psi=TRUNC_VAL)
    latent_img = Image.fromarray(images).resize((results_size, results_size))
    
    latent_img_output = image_outputs.children[1]
    with latent_img_output:
        latent_img_output.clear_output()
        display(latent_img)

def reset_latent_controls(self):
    
    image_outputs = controller.children[0]
    feature_sliders = controller.children[1]
    
    slider_hboxes = feature_sliders.children[:-2]
    for x in slider_hboxes:
        x.children[0].value = 0
        
    latent_img_output = image_outputs.children[1]
    with latent_img_output:
        latent_img_output.clear_output()
        display(image_to_use)

def create_interactive_latent_controller():
    orig_img_output = widgets.Output()

    with orig_img_output:
        orig_img_output.clear_output()
        display(image_to_use)

    latent_img_output = widgets.Output()

    with latent_img_output:
        latent_img_output.clear_output()
        display(image_to_use)

    image_outputs = widgets.VBox([orig_img_output, latent_img_output])

    #collapse-hide
    generate_button = widgets.Button(description='Generate', layout=widgets.Layout(width='75%', height='10%'))
    generate_button.on_click(apply_latent_controls)

    reset_button = widgets.Button(description='Reset Controls', layout=widgets.Layout(width='75%', height='10%'))
    reset_button.on_click(reset_latent_controls)

    feature_sliders = []
    for feature in latent_controls:
        label = widgets.Label(feature)
        slider = widgets.FloatSlider(min=-20, max=20)
        feature_sliders.append(widgets.HBox([slider, label]))
    feature_sliders.append(generate_button)
    feature_sliders.append(reset_button)
    feature_sliders = widgets.VBox(feature_sliders)

    return widgets.HBox([image_outputs, feature_sliders])

image_to_use = result_image
controller = create_interactive_latent_controller()
controller

#Logos with Text
This model generates logos with (illegible) text.

In [ ]:
#@title Download Requirements
print("Downloading Model")
print()
!gdown --id 124qoiGpbzS6p8UJVEDJ01uxPIF3uUYbV -O /content/model.pkl

print("Loading Directions")
print()
!gdown --id 10ffU99VmrILak65Wtf_i6_Sl0R50sWjB -O /content/directions.pkl

print("Cloning Custom StyleGAN2 Repository")
#@title Load In Model
!git clone https://github.com/arin-champati/stylegan2
%cd stylegan2/
import sys, os
%tensorflow_version 1.x

import pretrained_networks
import dnnlib
from dnnlib import tflib
import matplotlib.pyplot as plt
from IPython.display import clear_output

from pathlib import Path
from PIL import Image
import pickle
import numpy as np

import ipywidgets as widgets
from tqdm import tqdm
import tensorflow as tf
import random
TRUNC_VAL = 0.7
from matplotlib.gridspec import SubplotSpec
def create_subtitle(fig: plt.Figure, grid: SubplotSpec, title: str):
    "Sign sets of subplots with title"
    row = fig.add_subplot(grid)
    # the '\n' is important
    row.set_title(f'{title}\n', fontweight='semibold')
    # hide subplot
    row.set_frame_on(False)
    row.axis('off')

def generate_neighbors_w(og_w, diameter, num_samples, start, end):
    ws = []
    ws.append(og_w)
    for s in range(num_samples):
        random = np.random.uniform(-diameter,diameter,[12,512])
        w = og_w.copy()
        for layer in range(start, end):
          w[layer] += random[layer]

        ws.append(w)
    return ws
def linear_interpolate(code1, code2, alpha):
    return code1 * alpha + code2 * (1 - alpha)

def linear_interpolate_w(w1, w2, shape_alpha, color_alpha):
  w1 = w1.squeeze()
  w2 = w2.squeeze()
  result_w = np.zeros(w1.shape)

  # shape features
  for layer in range(0, 7):
      result_w[layer] = w2[layer] * shape_alpha + w1[layer] * (1 - shape_alpha)
  
  for layer in range(7, 12):
      result_w[layer] = w2[layer] * color_alpha + w1[layer] * (1 - color_alpha)

  return result_w

# Generate images given a random seed (Integer)
def generate_image_random(rand_seed):
    rnd = np.random.RandomState(rand_seed)
    z = rnd.randn(1, *Gs.input_shape[1:])
    tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars})
    images = Gs.run(z, None, **Gs_kwargs)
    return images[0], z

# Generate images given a latent code ( vector of size [1, 512] )
def generate_image_from_z(z):
    images = Gs.run(z, None, **Gs_kwargs)
    return images[0]
def z_to_w(z):
    w = Gs.components.mapping.run(z, None)[0]
    return w
def truncate(dlatents, truncation_psi, maxlayer = 12):
    dlatent_avg = tf.get_default_session().run(Gs.own_vars["dlatent_avg"])
    layer_idx = np.arange(12)[np.newaxis, :, np.newaxis]
    ones = np.ones(layer_idx.shape, dtype=np.float32)
    coefs = tf.where(layer_idx < maxlayer, truncation_psi * ones, ones)
    return tf.get_default_session().run(tflib.lerp(dlatent_avg, dlatents, coefs))
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols
    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid
# Code to load the StyleGAN2 Model
def generate_image_from_w(dlatents, truncation_psi = TRUNC_VAL, randomize_noise = False):
    if not truncation_psi is None:
        dlatents_trunc = truncate(dlatents, truncation_psi)
    else:
        dlatents_trunc = dlatents
        
    # Run the network
    fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    result_image = Gs.components.synthesis.run(
        dlatents_trunc.reshape((-1, 12, 512)),
        randomize_noise = randomize_noise,
        minibatch_size = 1,
        output_transform=fmt
    )[0]
    return result_image

def generate_neighbors(og_z, diameter, num_samples):
    zs = []
    zs.append(og_z)
    for s in range(num_samples):
        # np.random.seed()
        random = np.random.uniform(-diameter,diameter,[1,512])
        zs.append(np.clip(np.add(og_z,random),-1,1))
    return zs
    
def load_model():
    _G, _D, Gs = pretrained_networks.load_networks('/content/model.pkl')
    
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False 
    Gs_kwargs.truncation_psi = TRUNC_VAL
    return Gs, noise_vars, Gs_kwargs
results_size = 128
Gs, noise_vars, Gs_kwargs = load_model()
good_seeds = [569951935, 800520321, 862018663, 481784625,724559729,290190677,819924677,75406382,913566690,149985655,891990358,406968043, 872218544, 329271695, 299303370, 820506139, 844236123, 10286195,121987775, 198181997,873813230,282612399,478614833,235607220,8987070,523185080,50487580,825642669,665143910,65896362,900225085,120110703,193423295,403519303]

print()
clear_output()
print("Ready!")

In [ ]:
#@title Would You Like A Random Logo Or Preselected Good First Logo? (Make Sure To Rerun the Cell After Selecting)
logo_method_1 = "Good" #@param ["Random", "Good"]

In [ ]:
#@title Generate First Logo
# Ask the generator to make an output, given a random seed number: 42
if logo_method_1 == 'Random':
  TRUNC_VAL = 0.7
  rand_seed = np.random.randint(0,1000000000)
  images1, z1 = generate_image_random(rand_seed)
  w1 = z_to_w(z1)
  image1 = Image.fromarray(images1).resize((results_size, results_size))
  display(image1)
elif logo_method_1 == 'Good':
  TRUNC_VAL = 0.7
  rand_seed = random.choice(good_seeds)
  images1, z1 = generate_image_random(rand_seed)
  w1 = z_to_w(z1)
  image1 = Image.fromarray(images1).resize((results_size, results_size))
  display(image1)

In [ ]:
from matplotlib import pyplot as plt
#@title Generate Similar Logos  
num_samples = 8
neighbors1 = generate_neighbors_w(w1, diameter=6, num_samples=num_samples, start=0, end=5)
neighbors2 = generate_neighbors_w(w1, diameter=5, num_samples=num_samples, start=8, end=12)
neighbors3 = generate_neighbors(z1, diameter=.5, num_samples=num_samples)

f,ax = plt.subplots(3,len(neighbors1),figsize=(13,6))

all_neighbors_1 = [neighbors1, neighbors2, neighbors3]
count = 0
row_col = []
for i, neighbors in enumerate([neighbors1, neighbors2, neighbors3]):
  for j, im in enumerate(neighbors):
    if i == 2:
      ax[i, j].imshow(generate_image_from_z(im))
    else:
      ax[i, j].imshow(generate_image_from_w(im, truncation_psi=TRUNC_VAL))

    ax[i, j].title.set_text(count)
    ax[i, j].axis('off')
    row_col.append((i, j))
    count += 1

grid = plt.GridSpec(3, len(neighbors1))
create_subtitle(f, grid[0, ::], 'Varying Shape')
create_subtitle(f, grid[1, ::], 'Varying Color')
create_subtitle(f, grid[2, ::], 'Varying Both')
f.tight_layout()
# plt.savefig('/content/drive/MyDrive/Junior Year/IW/Code/figures/similar_logos/apple_regular_projection.png')
plt.show()

In [ ]:
#@title Select First Logo from Above Choices
first_index = int(input('Which logo would you like to proceed with? (Enter a number): '))
row = row_col[first_index][0]
col = row_col[first_index][1]

w1 = all_neighbors_1[row][col]
if w1.shape[0] != 12:
  w1 = z_to_w(w1)

images1 = generate_image_from_w(w1,truncation_psi=TRUNC_VAL).squeeze()
image1 = Image.fromarray(images1).resize((results_size, results_size))
display(image1)

In [ ]:
#@title Would You Like A Random Logo Or Preselected Good Second Logo? (Make Sure To Rerun the Cell After Selecting)
logo_method_2 = "Good" #@param ["Random", "Good"]

In [ ]:
#@title Generate Second Logo
if logo_method_2 == 'Random':
  TRUNC_VAL = 0.7
  rand_seed = np.random.randint(0,1000000000)
  images2, z2 = generate_image_random(rand_seed)
  w2 = z_to_w(z2)
  image2 = Image.fromarray(images2).resize((results_size, results_size))
  display(image2)
elif logo_method_2 == 'Good':
  TRUNC_VAL = 0.7
  rand_seed = random.choice(good_seeds)
  images2, z2 = generate_image_random(rand_seed)
  w2 = z_to_w(z2)
  image2 = Image.fromarray(images2).resize((results_size, results_size))
  display(image2)

In [ ]:
from matplotlib import pyplot as plt
#@title Generate Similar Logos  
num_samples = 8
neighbors1 = generate_neighbors_w(w2, diameter=6, num_samples=num_samples, start=0, end=5)
neighbors2 = generate_neighbors_w(w2, diameter=5, num_samples=num_samples, start=8, end=12)
neighbors3 = generate_neighbors(z2, diameter=.5, num_samples=num_samples)

f,ax = plt.subplots(3,len(neighbors1),figsize=(13,6))

all_neighbors_2 = [neighbors1, neighbors2, neighbors3]
count = 0
row_col = []
for i, neighbors in enumerate([neighbors1, neighbors2, neighbors3]):
  for j, im in enumerate(neighbors):
    if i == 2:
      ax[i, j].imshow(generate_image_from_z(im))
    else:
      ax[i, j].imshow(generate_image_from_w(im, truncation_psi=TRUNC_VAL))

    ax[i, j].title.set_text(count)
    ax[i, j].axis('off')
    row_col.append((i, j))
    count += 1

grid = plt.GridSpec(3, len(neighbors1))
create_subtitle(f, grid[0, ::], 'Varying Shape')
create_subtitle(f, grid[1, ::], 'Varying Color')
create_subtitle(f, grid[2, ::], 'Varying Both')
f.tight_layout()
# plt.savefig('/content/drive/MyDrive/Junior Year/IW/Code/figures/similar_logos/apple_regular_projection.png')
plt.show()

In [ ]:
#@title Select Second Logo from Above Choices
second_index = int(input('Which logo would you like to proceed with? (Enter a number): '))
row = row_col[second_index][0]
col = row_col[second_index][1]

w2 = all_neighbors_2[row][col]
if w2.shape[0] != 12:
  w2 = z_to_w(w2)

images2 = generate_image_from_w(w2,truncation_psi=TRUNC_VAL).squeeze()
image2 = Image.fromarray(images2).resize((results_size, results_size))
display(image2)

In [ ]:
#@title Mix Styles
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
def f(shape_alpha, color_alpha):
    images = generate_image_from_w(linear_interpolate_w(w1, w2, shape_alpha, color_alpha),truncation_psi=TRUNC_VAL)
    images = Image.fromarray(images).resize((results_size, results_size))
    im_grid = image_grid([image1, images, image2], 1, 3)
    return im_grid
shape_alpha = widgets.FloatSlider(min=0, max=1, step=.05, value=0, description= 'shape')
color_alpha = widgets.FloatSlider(min=0, max=1, step=.05, value=0, description= 'color')
current_im = interact(f, shape_alpha=shape_alpha, color_alpha=color_alpha);

In [ ]:
#@title Modify Color
latent_code_to_use = linear_interpolate_w(w1, w2, shape_alpha.value, color_alpha.value)
pkl_file = open('/content/directions.pkl', 'rb')
latent_controls = pickle.load(pkl_file)
pkl_file.close()
for key in latent_controls:
  latent_controls[key][3] = np.squeeze(latent_controls[key][3])
result_image = generate_image_from_w(latent_code_to_use)
result_image = Image.fromarray(result_image)
def apply_latent_controls(self):
    
    image_outputs = controller.children[0]
    feature_sliders = controller.children[1]
    
    slider_hboxes = feature_sliders.children[:-2]
    latent_movements = [(x.children[1].value, x.children[0].value) for x in slider_hboxes]

    modified_latent_code = np.array(latent_code_to_use)
    for feature, amount_to_move in latent_movements:
      for layer in range(latent_controls[feature][0],latent_controls[feature][1]):
        modified_latent_code[layer] += latent_controls[feature][3]*amount_to_move*latent_controls[feature][2]

    images = generate_image_from_w(modified_latent_code)
    latent_img = Image.fromarray(images).resize((results_size, results_size))
    
    latent_img_output = image_outputs.children[1]
    with latent_img_output:
        latent_img_output.clear_output()
        display(latent_img)

def reset_latent_controls(self):
    
    image_outputs = controller.children[0]
    feature_sliders = controller.children[1]
    
    slider_hboxes = feature_sliders.children[:-2]
    for x in slider_hboxes:
        x.children[0].value = 0
        
    latent_img_output = image_outputs.children[1]
    with latent_img_output:
        latent_img_output.clear_output()
        display(image_to_use)

def create_interactive_latent_controller():
    orig_img_output = widgets.Output()

    with orig_img_output:
        orig_img_output.clear_output()
        display(image_to_use)

    latent_img_output = widgets.Output()

    with latent_img_output:
        latent_img_output.clear_output()
        display(image_to_use)

    image_outputs = widgets.VBox([orig_img_output, latent_img_output])

    #collapse-hide
    generate_button = widgets.Button(description='Generate', layout=widgets.Layout(width='75%', height='10%'))
    generate_button.on_click(apply_latent_controls)

    reset_button = widgets.Button(description='Reset Latent Controls', layout=widgets.Layout(width='75%', height='10%'))
    reset_button.on_click(reset_latent_controls)

    feature_sliders = []
    for feature in latent_controls:
        label = widgets.Label(feature)
        slider = widgets.FloatSlider(min=-20, max=20)
        feature_sliders.append(widgets.HBox([slider, label]))
    feature_sliders.append(generate_button)
    feature_sliders.append(reset_button)
    feature_sliders = widgets.VBox(feature_sliders)

    return widgets.HBox([image_outputs, feature_sliders])
image_to_use = result_image
controller = create_interactive_latent_controller()
controller